# Homework 2

## Task 1: Tropical Max-Plus Algebra

# What are the outputs of the following expressions?

In [1]:
using TropicalNumbers
println(Tropical(1.0) + Tropical(3.0))
println(one(Tropical{Float64}))
println(zero(Tropical{Float64}))

3.0ₜ
0.0ₜ
-Infₜ


What is the type and supertype of Tropical(1.0)?


In [2]:
println(typeof(Tropical(1.0)))
println(supertype(Tropical{Float64}))

Tropical{Float64}
AbstractSemiring
Float64}
AbstractSemiring


Is Tropical a concrete type or an abstract type?

In [3]:
isconcretetype(Tropical)

false

In [4]:
isabstracttype(Tropical)

false

Is Tropical{Real} a concrete type or an abstract type?

In [5]:
isconcretetype(Tropical{Real})

true

In [6]:
isabstracttype(Tropical{Real})

false

Benchmark and profile the performance of Tropical matrix multiplication:

In [9]:
using Random
using BenchmarkTools
using Profile

function Base.rand(::Type{Tropical{T}}, rng::AbstractRNG) where T
    return Tropical(rand(rng, T))  
end

function Base.rand(::Type{Tropical{T}}, dims::Tuple{Vararg{Int}}) where T
    rng = Random.TaskLocalRNG()  # 使用默认的随机数生成器
    # 生成一个一维数组，然后将其重塑为指定的维度
    return reshape([rand(Tropical{T}, rng) for _ in 1:prod(dims)], dims)
end


A = rand(Tropical{Float64}, 100, 100)
B = rand(Tropical{Float64}, 100, 100)


@btime C = A * B # 2.066 ms
@profile for i in 1:10; A * B; end
Profile.print(format=:flat,mincount=10)


  1.499 ms (3 allocations: 78.20 KiB)
 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
    53         0 @Base/Base.jl            557 include(mod::Module, _path::Strin…
    14        14 @Base/array.jl           994 setindex!
    53         0 @Base/boot.jl            430 eval
    53         0 @Base/client.jl          531 _start()
    53         0 @Base/client.jl          323 exec_options(opts::Base.JLOptions)
    53         0 @Base/essentials.jl     1055 #invokelatest#2
    17        17 @Base/essentials.jl      796 ifelse
    53         0 @Base/essentials.jl     1052 invokelatest
    53         0 @Base/loading.jl        2794 _include(mapexpr::Function, mod::…
    53         0 @Base/loading.jl        2734 include_string(mapexpr::typeof(id…
    12         0 @Base/math.jl            839 max
    22         0 @Base/promotion.jl       633 muladd
    45         0 @Base/simdloop.jl         77 macro expansion
    53         0 …Algebra/s

# Task 2: Implement the the following semiring algebra over sets:

In [12]:
struct SetSemiring
    elements::Set{Int}
    
    # 外部构造函数：允许通过多个整数构造（如 SetSemiring(1, 2, 3)）
    SetSemiring(args::Integer...) = new(Set(args))
    
    # 内部构造函数：直接接受 Set{Int} 类型（用于运算结果）
    SetSemiring(s::Set{Int}) = new(s)
end

# 加法：集合的并集
Base.:(+)(a::SetSemiring, b::SetSemiring) = SetSemiring(union(a.elements, b.elements))

# 乘法：所有元素乘积的集合
Base.:(*)(a::SetSemiring, b::SetSemiring) = SetSemiring(Set(x * y for x in a.elements for y in b.elements))

# 零元：空集
Base.zero(::Type{SetSemiring}) = SetSemiring(Set{Int}())

# 单位元：包含1的集合
Base.one(::Type{SetSemiring}) = SetSemiring(1)

# 相等性判断
Base.:(==)(a::SetSemiring, b::SetSemiring) = a.elements == b.elements

# 自定义显示格式
Base.show(io::IO, s::SetSemiring) = print(io, "SetSemiring($(s.elements))")

test code like this

In [13]:
using Test

@testset "SetSemiring Tests" begin
    # 加法测试
    @test SetSemiring(2) + SetSemiring(5, 4) == SetSemiring(2, 4, 5)
    @test SetSemiring(5, 4) + SetSemiring(2) == SetSemiring(2, 4, 5)
    @test SetSemiring(2) + zero(SetSemiring) == SetSemiring(2)
    
    # 乘法测试
    @test SetSemiring(2) * SetSemiring(5, 4) == SetSemiring(10, 8)
    @test SetSemiring(5, 4) * SetSemiring(2) == SetSemiring(10, 8)
    @test SetSemiring(2) * zero(SetSemiring) == zero(SetSemiring)
    @test zero(SetSemiring) * SetSemiring(2) == zero(SetSemiring)
    @test SetSemiring(2) * one(SetSemiring) == SetSemiring(2)
    @test one(SetSemiring) * SetSemiring(2) == SetSemiring(2)
    
    # 零元和单位元测试
    @test zero(SetSemiring) == SetSemiring()
    @test one(SetSemiring) == SetSemiring(1)
end

Test Summary:     | Pass  Total  Time
SetSemiring Tests |   11     11  0.1s


Test.DefaultTestSet("SetSemiring Tests", Any[], 11, false, false, true, 1.740988775826436e9, 1.740988775975418e9, false, "/hpc2hdd/home/chuo657/work-tjh/AMAT5315/AMAT5315-2025Spring-Homeworks/hw2/JiahuTANG/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X33sdnNjb2RlLXJlbW90ZQ==.jl")